In [2]:
import sys
sys.path.append('/Users/samiraabnar/Codes/GoogleLM1b/')
sys.path.append('/Users/samiraabnar/Codes/Bridge/')

from ExplainBrain import ExplainBrain
from read_dataset.harrypotter_data import HarryPotterReader
from computational_model.text_encoder import TfHubElmoEncoder, TfTokenEncoder, TfHubLargeUniversalEncoder, GloVeEncoder, GoogleLMEncoder
from mapping_models.sk_mapper import SkMapper

import tensorflow as tf
import numpy as np
import os

In [8]:
brain_data_dir = '/Users/samiraabnar/Codes/Data/harrypotter/'
subject_ids = [1]
brain_data_reader = HarryPotterReader(data_dir=brain_data_dir)


In [9]:
all_events = brain_data_reader.read_all_events(subject_ids=subject_ids)

Reading /Users/samiraabnar/Codes/Data/harrypotter/subject_1.mat
Reading /Users/samiraabnar/Codes/Data/harrypotter/subject_1.mat
Reading /Users/samiraabnar/Codes/Data/harrypotter/subject_1.mat
Reading /Users/samiraabnar/Codes/Data/harrypotter/subject_1.mat


In [75]:
number_of_sentences = {}
sentences_length = {}
subject_id = 1
for block in all_events[subject_id]:
    number_of_sentences[block.block_id] = len(block.sentences)
    sentences_length[block.block_id] = []
    for sent in block.sentences:
        sentences_length[block.block_id].append(len(sent))
    
print("number of sentences per block: ", number_of_sentences)  
print("average sentence length per block")
for block in [1,2,3,4]:
    print("block ", block,": ",np.median(sentences_length[block]),np.mean(sentences_length[block]), min(sentences_length[block]), max(sentences_length[block]))
    print("sentences larger than 10:", len(np.where(np.asarray(sentences_length[block]) > 10)[0]))

number of sentences per block:  {1: 115, 2: 163, 3: 134, 4: 177}
average sentence length per block
block  1 :  10.0 11.330434782608696 1 44
sentences larger than 10: 56
block  2 :  6.0 8.28834355828221 1 74
sentences larger than 10: 48
block  3 :  5.0 7.902985074626866 1 36
sentences larger than 10: 35
block  4 :  5.0 8.265536723163843 1 80
sentences larger than 10: 53


In [76]:
stimuli = np.load('/Users/samiraabnar/Codes/bridge_data/processed/harrypotter/1_none_window0-0_stimuli_in_context.npy').item()
time_steps = np.load('/Users/samiraabnar/Codes/bridge_data/processed/harrypotter/1_none_window0-0_time_steps.npy').item()


In [77]:
stimuli_dic_per_block = {1:{},2:{},3:{},4:{}}
stimuli_dic_general = {}
for block in [1,2,3,4]:
    for stim, index in  stimuli[block]:
        stim = ' '.join(stim)
        if index is not None:
            if not stim in stimuli_dic_per_block[block]:
                stimuli_dic_per_block[block][stim] = 0
            if not stim in stimuli_dic_general:
                stimuli_dic_general[stim] = 0
            stimuli_dic_per_block[block][stim] += 1
            stimuli_dic_general[stim] += 1                       

In [94]:
print("Unique stimuli and total number of stimuli")
print(len(stimuli_dic_general.keys()), sum(stimuli_dic_general.values()))
for block in [1,2,3,4]:
    print("block ",block,": ",len(stimuli_dic_per_block[block].keys()), sum(stimuli_dic_per_block[block].values()))
   
for stim, count in stimuli_dic_general.items():
    if count > 1:
        print(stim, count)

Unique stimuli and total number of stimuli
1292 1295
block  1 :  326 326
block  2 :  338 338
block  3 :  265 265
block  4 :  366 366
on the ground and 2
the portrait of the 2
+ It was a 2


In [79]:
full_stimuli = []
full_steps = []
full_blocks = []
for block in [1,2,3,4]:
  for stim_in_c, step in zip(stimuli[block],time_steps[block]):
    stim = list(np.asarray(stim_in_c[0])[0 if stim_in_c[1] is None else stim_in_c[1]])
    if len(stim) == 0:
      stim =[' ']
    full_stimuli.extend(stim)
    full_steps.extend([step]*len(stim))
    full_blocks.extend([block]*len(stim))

In [84]:
vocab_general = {}
vocab_per_block = {1:{},2:{},3:{},4:{}}

for token,block in zip(full_stimuli,full_blocks):
    if len(token.strip()) > 0:
        if token not in vocab_general:
            vocab_general[token] = 0
        if token not in vocab_per_block[block]:
            vocab_per_block[block][token] = 0
        vocab_general[token] += 1  
        vocab_per_block[block][token] += 1

In [85]:
print("total vocab size:", len(vocab_general.keys()))
print("total number of words:", sum(vocab_general.values()))

for block in [1,2,3,4]:
    print("block ",block,": ",len(vocab_per_block[block].keys()), sum(vocab_per_block[block].values()))


total vocab size: 1362
total number of words: 6558
block  1 :  553 1583
block  2 :  560 1711
block  3 :  461 1411
block  4 :  583 1853


In [93]:
print("how many words occur only once:")
print(len(np.where(np.asarray(list(vocab_general.values())) == 1)[0]))

print("how many words occur more than twice:")
print(len(np.where(np.asarray(list(vocab_general.values())) > 2)[0]))

how many words occur only once:
779
how many words occur more than twice:
348
